Выполнил: Драгомирский Даглар Сарматович

Группа: РИМ-220963

### ДОМАШНЕЕ ЗАДАНИЕ №2
В первой части проекта вы реализовывали классификацию при помощи эвристик и методов классического ML поверх векторизованных слов.

В этой части нужно решить эту же задачу при помощи RNN.



#### ЗАДАЧА
Обучите несколько моделей рекуррентных нейронных сетей, например LSTM, GRU, Bidirectional-LSTM.
Посчитайте значение метрики, которую вы предложили в части 1 и сравните результаты для разных RNN, эвристик и классического ML.


#### РЕЗУЛЬТАТ:
Jupyter-notebook с реализацией требуемых методов.

Эту часть проекта мы будем оценивать по тем же критериям:

Полнота выполненной работы;
Общее качество кода и следование PEP-8;
Итоговое значение метрики качества.


#### ПОДСКАЗКИ:
При работе с RNN можно экспериментировать не только с типом слоя, но и с гиперпараметрами.
«Сравнить результаты» означает не просто посчитать метрику для разных методов, но и сделать вывод о том, что сработало лучше
Не забудьте реализовать валидационную выборку и на ней отследить, что модель не переобучилась.

# ВАЖНАЯ ИНФА

1. При решении д\з с использованием  PyTorch столкнулся со значительными трудностями. В итоге было принято решение использовать tensorflow
2. Используем уже обработанный датасет из 1-го д\з

# Подключение необходимых библиотек

In [2]:
import os
from pathlib import Path
from typing import Literal

import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from tensorflow.keras import Sequential, layers
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer


## CONSTANTS

In [37]:

#MODELS_FOLDER = os.path.join(DATA_FOLDER, "models")
#Path(MODELS_FOLDER).mkdir(parents=True, exist_ok=True)
PREPROCESSED_CSV = os.path.join("PREPROCESSED.csv")
EVAL_CSV = os.path.join("evaluate.csv")
EVAL_CSV_NEW = os.path.join("evaluate_new.csv")
EVAL_CSV_RATING = os.path.join("evaluate_rating.csv")

# Загрузка данных

In [23]:
# Используем уже обработанный датасет из 1-го д\з:
preprocessed_df = pd.read_csv(PREPROCESSED_CSV)
preprocessed_df = preprocessed_df.dropna().reset_index(drop=True)
preprocessed_df

,text,sentiment,text_clean,text_stemmer,text_lemma,text_split,text_symbols_n,text_list_len,text_set_len
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,3,menyrbie philgahan chrisitv,menyrbi philgahan chrisitv,menyrbie philgahan chrisitv,"['menyrbie', 'philgahan', 'chrisitv']",30,3,3
1,advice Talk to your neighbours family to excha...,4,advice talk neighbours family exchange phone n...,advic talk neighbour famili exchang phone numb...,advice talk neighbour family exchange phone nu...,"['advice', 'talk', 'neighbours', 'family', 'ex...",196,27,24
2,Coronavirus Australia: Woolworths to give elde...,4,coronavirus australia woolworths give elderly...,coronavirus australia woolworth give elder dis...,coronavirus australia woolworth give elderly d...,"['coronavirus', 'australia', 'woolworths', 'gi...",101,12,12
3,My food stock is not the only one which is emp...,4,food stock one empty please nt panic enough...,food stock one empti pleas nt panic enough foo...,food stock one empty please nt panic enough fo...,"['food', 'stock', 'one', 'empty', 'please', 'n...",181,23,20
4,"Me, ready to go at supermarket during the #COV...",1,ready go supermarket covid outbreak m parano...,readi go supermarket covid outbreak m paranoid...,ready go supermarket covid outbreak m paranoid...,"['ready', 'go', 'supermarket', 'covid', 'outbr...",198,23,23
...,...,...,...,...,...,...,...,...,...
41135,Airline pilots offering to stock supermarket s...,3,airline pilots offering stock supermarket shel...,airlin pilot offer stock supermarket shelv nz ...,airline pilot offering stock supermarket shelf...,"['airline', 'pilots', 'offering', 'stock', 'su...",69,9,9
41136,Response to complaint not provided citing COVI...,1,response complaint provided citing covid relat...,respons complaint provid cite covid relat dela...,response complaint provided citing covid relat...,"['response', 'complaint', 'provided', 'citing'...",109,16,16
41137,You know itÂs getting tough when @KameronWild...,4,know getting tough kameronwilds rationing toi...,know get tough kameronwild ration toilet paper...,know getting tough kameronwilds rationing toil...,"['know', 'getting', 'tough', 'kameronwilds', '...",113,13,13
41138,Is it wrong that the smell of hand sanitizer i...,3,wrong smell hand sanitizer starting turn cor...,wrong smell hand sanit start turn coronavirus ...,wrong smell hand sanitizer starting turn coron...,"['wrong', 'smell', 'hand', 'sanitizer', 'start...",74,9,8


In [24]:
# Так же используем таблицу с оценками способов предсказания:
eval_df = pd.read_csv(EVAL_CSV)
eval_df

,method,accuracy
0,random,0.192929
1,most_popular,0.279917
2,unique_words,0.301664
3,multinomial_naive_bayes,0.444296
4,random_forest_classifier,0.527153


# Подготовка данных

Масштабируем оценки [1; 5] (где 1-негативно, 5-позитивно) -> в диапазон [0; 4]:

In [25]:
preprocessed_df["sentiment"] = preprocessed_df["sentiment"].apply(lambda x: x - 1)

Разделим на:
- предикторы
- target (собственно оценку текста):

In [26]:
X = preprocessed_df["text_clean"]
y = preprocessed_df["sentiment"].values

In [27]:
MAX_LEN = 50

# Токенизация текста
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

# паддинг
X = pad_sequences(X, padding='post', maxlen=MAX_LEN)
MAX_VOCAB_SIZE = len(tokenizer.word_index)
print(f"Vocabulary size = {MAX_VOCAB_SIZE}")
print(f"X.shape = {X.shape}")

Vocabulary size = 55648
X.shape = (41140, 50)


Токенизированный текст -> train + test:

In [28]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    stratify=y,
    test_size=0.1,
    random_state=42,
)
X_train[4]

array([  216,   881,   802,     2,  1470, 23569,  2571,   571, 11090,
       23570,   789,   116,    26,     7,    54,   171,   175,  2103,
         114,  8857,  7468,   719,   818,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0])

In [29]:
# Ключевые константы + гиперпараметры:
EMBEDDING_DIM = 16
HIDDEN_DIM = 256
OUTPUT_SIZE = 5
VOCAB_SIZE = MAX_VOCAB_SIZE + 1
BATCH_SIZE = 16
LEARNING_RATE = 0.001
NUM_EPOCHS = 3
DROPOUT = 0.3
VALIDATION_SPLIT = 0.1

In [32]:
# Зададим класс MyNlp, в рамках которого будем задавать одну из нейросеток (GRU, LSTM, BDLSTM) -> активацию в главном слое -> обучение + тестирование модели на наших test/train данных:
class MyNlp:
    def __init__(
        self,
        model_type: Literal["GRU", "LSTM", "BDLSTM"],
        activation: str,
        epoch: int = NUM_EPOCHS,
        hidden_dim: int = HIDDEN_DIM,
        vocab_size: int = VOCAB_SIZE,
        embedding_dim: int = EMBEDDING_DIM,
        input_length: int = MAX_LEN,
        output_size: int = OUTPUT_SIZE,
        batch_size: int = BATCH_SIZE,
        dropout: float = DROPOUT,
        validation_split: float = VALIDATION_SPLIT,
    ) -> None:
        self.model_name = f"{model_type}_{activation}"
        if model_type == "GRU":
            self.main_layer = layers.GRU(
                hidden_dim,
                return_sequences=True,
            )
        elif model_type == "LSTM":
            self.main_layer = layers.LSTM(
                hidden_dim,
                return_sequences=True,
            )
        elif model_type == "BDLSTM":
            self.main_layer = layers.Bidirectional(
                layers.LSTM(
                    hidden_dim,
                    return_sequences=True,
                )
            )
        else:
            raise ValueError('`model_type` should be one of "GRU", "LSTM", "BDLSTM"')
        self.model = Sequential(
            [
                layers.Embedding(
                    vocab_size,
                    embedding_dim,
                    input_length=input_length,
                ),
                self.main_layer,  # Main nlp layer depending on model type
                layers.GlobalMaxPool1D(),
                layers.Dropout(dropout),
                layers.Dense(
                    64,
                    activation=activation,
                ),
                layers.Dropout(dropout),
                layers.Dense(output_size),
            ],
            name=self.model_name,
        )
        self.optimizer = Adam()
        self.epoch = epoch
        self.validation_split = validation_split
        self.batch_size = batch_size

    def _print_line(self, item: str = "_") -> None:
        print(item * 65)

    def compile(self) -> None:
        self.model.compile(
            loss=SparseCategoricalCrossentropy(from_logits=True),
            optimizer=self.optimizer,
            metrics=["accuracy"],
        )
        self.model.summary()
        print("\n\n")

    def train(self, X, y) -> None:
        print(f"Start training model `{self.model_name}`:")
        self._print_line()
        self.history = self.model.fit(
            X,
            y,
            epochs=self.epoch,
            validation_split=self.validation_split,
            batch_size=self.batch_size,
        )
        self._print_line()
        print("\n\n")

    def test(self, X, y) -> None:
        print(f"Start testing model `{self.model_name}`:")
        self._print_line()
        y_predicted = np.argmax(
            self.model.predict(X),
            axis=1,
        )
        self.accuracy = accuracy_score(y_predicted, y)
        print(f"Test Accuracy: {round(self.accuracy, 6)}")
        self._print_line("**")
        print("\n\n\n")

    def add_info_to_df(self, df) -> None:
        df.loc[len(df)] = [
            self.model_name,
            self.accuracy,
        ]

    def save_model(self) -> None:
        self.model.save(os.path.join(f"{self.model_name}.keras"))

    def train_test_pipeline(
        self,
        X_train,
        y_train,
        X_test,
        y_test,
        df,
    ) -> None:
        self.compile()
        self.train(X_train, y_train)
        self.test(X_test, y_test)
        self.add_info_to_df(df)
        self.save_model()

Обучаем 3 нейросети -> для каждой 3 типа активации (elu, relu, leaky_relu):

In [33]:
for model_type in ["GRU", "LSTM", "BDLSTM"]:
    for activation in ["elu", "relu", "leaky_relu"]:
        model = MyNlp(model_type, activation)
        model.train_test_pipeline(X_train, y_train, X_test, y_test, eval_df)

Model: "GRU_elu"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 16)            890384    
                                                                 
 gru_1 (GRU)                 (None, 50, 256)           210432    
                                                                 
 global_max_pooling1d_1 (Glo  (None, 256)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                16448     
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                           

In [36]:
# Результаты
eval_df.to_csv(EVAL_CSV_NEW, index=False)
eval_df

,method,accuracy
0,random,0.192929
1,most_popular,0.279917
2,unique_words,0.301664
3,multinomial_naive_bayes,0.444296
4,random_forest_classifier,0.527153
5,GRU_elu,0.736266
6,GRU_elu,0.721682
7,GRU_relu,0.718036
8,GRU_leaky_relu,0.722654
9,LSTM_elu,0.721682


In [38]:
# Сортировка + рейтинг моделек:
eval_df_sorted = eval_df.sort_values(by="accuracy", ascending=False).reset_index(drop=True)
eval_df_sorted.to_csv(EVAL_CSV_RATING, index=False)
eval_df_sorted

,method,accuracy
0,BDLSTM_leaky_relu,0.743802
1,GRU_elu,0.736266
2,BDLSTM_relu,0.734079
3,LSTM_relu,0.731891
4,BDLSTM_elu,0.730433
5,GRU_leaky_relu,0.722654
6,GRU_elu,0.721682
7,LSTM_elu,0.721682
8,GRU_relu,0.718036
9,LSTM_leaky_relu,0.716334


# Вывод

Результаты моделей, на основе нейросетей показали примерно одинаковый результат (точность более 70%) что значительно лучше тех же моделей classic ML.
При прогоне несколько раз - результаты могут отличаться, но в среднем все справляются нормально. (Разве что GRU и leaky_relu чаще других выбивались в лидеры).